In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pandas as pd
import numpy as np
from gensim import corpora

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
skillXproblem = pd.read_csv('./skillXproblem.csv')
print 'CSV file shape:', skillXproblem.shape

columns = skillXproblem.columns.values
print columns 
stoplist = set('and &'.split())

texts = [[word for word in column.lower().split() if word not in stoplist]
        for column in columns]
print texts

CSV file shape: (96, 12)
['Problem' 'Electricity and Electronics' 'Behavioral & Social Sciences'
 'Biology' 'Chemistry' 'Computer Science & Math'
 'Earth & Environmental Studies' 'Energy & Space' 'Fauna & Flora'
 'Food Science' 'Inventions & Innovation' 'Physics']
[['problem'], ['electricity', 'electronics'], ['behavioral', 'social', 'sciences'], ['biology'], ['chemistry'], ['computer', 'science', 'math'], ['earth', 'environmental', 'studies'], ['energy', 'space'], ['fauna', 'flora'], ['food', 'science'], ['inventions', 'innovation'], ['physics']]
